基於 4chan 魔改版 NovelAILeaks (naifu) 製作。 [來源](https://boards.4channel.org/g/thread/89095460#p89097704)

使用官方前端 + 優化版後端，可突破75限制，支持所有模型。

In [1]:
!ls -lR

In [4]:
# Load the Drive helper and mount
from google.colab import drive
 
# This will prompt for authorization.
drive.mount('/content/drive')

In [6]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
from PIL import Image
import numpy as np
import cv2
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from google.colab.patches import cv2_imshow

transform1 = transforms.Compose([transforms.Resize([22, 22]), transforms.ToTensor()])


class FaceLandmarkDataset(Dataset):
    def __len__(self) -> int:
        print("!!!!!!!!", len(self.landmarks_frame))
        return len(self.landmarks_frame)

    def __init__(self, csv_file: str, root_dir: str, transform=None) -> None:
        super().__init__()
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __getitem__(self, index: int):
        img_name = self.landmarks_frame.Filename[index]
        print(img_name)
        img_path = os.path.join('/content/drive/MyDrive/tmp/', str(img_name) + ".png")
        print(img_path)
        # with Image.open(img_path) as img: 
        #     image = img.convert('RGB') 
        img = cv2.imread(img_path)

        PIL_image = Image.fromarray(img)  # 这里ndarray_image为原来的numpy数组类型的输入

        img1 = transform1(PIL_image)
        img2 = img1.numpy() * 255
        img2 = img2.astype('uint8')
        img2 = np.transpose(img2, (1, 2, 0))
        img2 = np.reshape(img2, (3, 22,22))
        landmarks = self.landmarks_frame.values[index, 1:].astype('float')
        print(landmarks)
        # landmarks = np.reshape(landmarks,newshape=(-1,2)) 
        # if self.transform is not None: 
        #     image = self.transform(image) 

        cv2_imshow(img2)
        # print(str(img2.size()))
        return img2, landmarks

    ########################以上为数据读取类（返回：image,landmarks）###############################


trans = transforms.Compose(transforms=[transforms.RandomResizedCrop(size=128),
                                       transforms.ToTensor()])

face_dataset = FaceLandmarkDataset(csv_file=r'/content/drive/MyDrive/tmp/Run History.csv',
                                   root_dir='/content/drive/MyDrive/tmp', transform=trans)
loader = DataLoader(dataset=face_dataset,
                    batch_size=4,
                    shuffle=True,
                    num_workers=4)

# img, label=face_dataset[1]
# cv2_imshow(img.numpy().transpose(1,2,0)*255)
# print(label)

# dataiterator = iter(loader)
# img, label = dataiterator.next()

# print(img.size(), label.size())

In [7]:
#@title ### 0. 檢查 GPU 工作狀態

!nvidia-smi

In [ ]:
#@title ### 1. 下載 Novel AI API 後端、模型 
#@markdown 如果下載速度太慢可以嘗試 restart

%cd /content/
!apt install -y -qq aria2
!aria2c --summary-interval=5 -x 3 --allow-overwrite=true -Z \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/naifu.tar \
  https://pub-2fdef7a2969f43289c42ac5ae3412fd4.r2.dev/animefull-latest.tar 
  
!echo "Decompressing..."
!tar xf naifu.tar && rm naifu.tar
!echo "Done."

In [ ]:
#@title ### 2. 安裝依賴
#@markdown 耐心等待安裝完成

%cd /content/naifu
!pip install virtualenv && bash ./setup.sh
!curl -Ls https://github.com/ekzhang/bore/releases/download/v0.4.0/bore-v0.4.0-x86_64-unknown-linux-musl.tar.gz | tar zx -C /usr/bin
!curl -Lo /usr/bin/cloudflared https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 && chmod +x /usr/bin/cloudflared

In [ ]:
#@title ### 3. 啟動模型
#@markdown 訪問輸出的映射地址（以 `trycloudflare.com` / `bore.pub` 結尾）即可。
#@markdown - 請等待模型加載完成（出現`Application startup complete`字樣）再訪問
#@markdown - cloudflare 提供的服務偶爾會出現請求超時，可換用 bore 隧道

%cd /content/naifu
!sed -i 's/pytorch_lightning.utilities.distributed/pytorch_lightning.utilities.rank_zero/g' /content/naifu/ldm/models/diffusion/ddpm.py
!sed -i 's/# export SAVE_FILES="1"/export SAVE_FILES="1"/g' run.sh
!bash run.sh & cloudflared tunnel --url localhost:6969

In [ ]:
#@title ### 4. （可選）使用 7G 的 `animefull-latest` 模型運行
#@markdown 默認使用的是 4G 大小的 animefull-final-pruned 模型。如果想使用 7G 的 animefull-latest 模型，運行這個

%cd /content/
!tar xf animefull-latest.tar -C /content/naifu/models && rm animefull-latest.tar
!sed -i 's/map_location="cpu"/map_location="cuda"/g' /content/naifu/hydra_node/models.py

%cd /content/naifu
%env DTYPE=float16
%env CLIP_CONTEXTS=3
%env AMP=1
%env MODEL=stable-diffusion
%env DEV=True
%env MODEL_PATH=models/animefull-latest
%env ENABLE_EMA=1
%env VAE_PATH=models/animevae.pt
%env PENULTIMATE=1
%env PYTHONDONTWRITEBYTECODE=1
%env SAVE_FILES=1

!./venv/bin/python -m uvicorn --host 0.0.0.0 --port=6969 main:app & bore local 6969 --to bore.pub & cloudflared tunnel --url localhost:6969

In [ ]:
!pip install virtualenv
!virtualenv venv
!cat /content/venv/bin/activate
tmpAct = '/content/venv/bin/activate'
!cp $tmpAct /content/venv/bin/activate1
!cp $tmpAct /content/venv/bin/activate2
!cp $tmpAct /content/venv/bin/activate3
!cp $tmpAct /content/venv/bin/activate4
!cp $tmpAct /content/venv/bin/activate5
!cp $tmpAct /content/venv/bin/activate6
!cp $tmpAct /content/venv/bin/activate7
!cp $tmpAct /content/venv/bin/activate8
!cp $tmpAct /content/venv/bin/activate9

In [ ]:
tmpAct = '/content/venv/bin/activate'

In [ ]:
fileName = tmpAct
!sed -i "s/export VIRTUAL_ENV_PROMPT/export  VIRTUAL_ENV_PROMPT \&\& echo \"os.environ['VIRTUAL_ENV_PROMPT'] = '\$VIRTUAL_ENV_PROMPT'\"/g" $fileName
!sed -i "s/export VIRTUAL_ENV/export VIRTUAL_ENV \&\& echo \"import os;\" \&\& echo \"os.environ['VIRTUAL_ENV'] = '\$VIRTUAL_ENV'\"/g" $fileName
!sed -i "s/export PS1/export PS1 \&\& echo \"os.environ['PS1'] = '\$PS1'\"/g" $fileName
!sed -i "s/export PATH/export PATH \&\& echo \"os.environ['PATH'] = '\$PATH'\"/g" $fileName
!sed -i "s/export PYTHONHOME/export PYTHONHOME \&\& echo \"os.environ['PYTHONHOME'] = '\$PYTHONHOME'\"/g" $fileName
!cat $tmpAct

In [ ]:
!. $tmpAct